# Продвинутое машинное обучение: ДЗ 2
### Герасимчик Анна. ML-22

In [1]:
import pandas as pd
import numpy as np
import pickle

import scipy as sp
from scipy import stats

from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder

import torch
from copy import deepcopy

## Описание

Второе домашнее задание — самое большое в курсе, в нём придётся и концептуально подумать о происходящем, и технические трудности тоже порешать. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 

Второе задание — это полноценный проект по анализу данных, начиная от анализа постановки задачи и заканчивая сравнением результатов разных моделей. Задача реальная и серьёзная, хотя тему я выбрал развлекательную: мы будем строить вероятностную рейтинг-систему для спортивного “Что? Где? Когда?” (ЧГК).

Background: в спортивном “Что? Где? Когда?” соревнующиеся команды отвечают на одни и те же вопросы. После минуты обсуждения команды записывают и сдают свои ответы на карточках; побеждает тот, кто ответил на большее число вопросов. Турнир обычно состоит из нескольких десятков вопросов (обычно 36 или 45, иногда 60, больше редко). Часто бывают синхронные турниры, когда на одни и те же вопросы отвечают команды на сотнях игровых площадок по всему миру, т.е. в одном турнире могут играть сотни, а то и тысячи команд. Соответственно, нам нужно:
- построить рейтинг-лист, который способен нетривиально предсказывать результаты будущих турниров;
- при этом, поскольку ЧГК — это хобби, и контрактов тут никаких нет, игроки постоянно переходят из команды в команду, сильный игрок может на один турнир сесть поиграть за другую команду и т.д.; поэтому единицей рейтинг-листа должна быть не команда, а отдельный игрок;
- а что сильно упрощает задачу и переводит её в область домашних заданий на EM-алгоритм — это характер данных: начиная с какого-то момента, в базу результатов начали вносить все повопросные результаты команд, т.е. в данных будут записи вида “какая команда на какой вопрос правильно ответила”.

Я сделал за вас только первый шаг: выкачал через API $\href{https://rating.chgk.info/}{сайта~рейтинга~ЧГК}$ все нужные данные, чтобы сайт не прилёг под вашими многочисленными скрейперами. :) Полученные данные лежат в формате pickle $\href{https://www.dropbox.com/s/s4qj0fpsn378m2i/chgk.zip}{вот~здесь}$.

### 1

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
- взять в тренировочный набор турниры с dateStart из 2019 года; 
- в тестовый — турниры с dateStart из 2020 года.

In [2]:
def load_data(data_path):
    with open(data_path + 'tournaments.pkl', 'rb') as tournaments_file, open(data_path + 'results.pkl', 'rb') as results_file, open(data_path + 'players.pkl', 'rb') as players_file:
        tournaments = pickle.load(tournaments_file)
        results = pickle.load(results_file)
        players = pickle.load(players_file)
    return tournaments, results, players

In [3]:
DATA_PATH = 'chgk/'
tournaments, results, players = load_data(DATA_PATH)

In [4]:
train_tournaments = [v['id'] for k, v in tournaments.items() if v['dateStart'][:4] == '2019']
test_tournaments = [v['id'] for k, v in tournaments.items() if v['dateStart'][:4] == '2020']

In [5]:
def get_team_names(results):
    teams_names = {}
    for tournament_id, tournament_teams in results.items():
        for team in tournament_teams:
            teams_names[team['team']['id']] = team['team']['name']  
    return teams_names

In [6]:
players_names = {v['id']: v['name'] + ' ' + v['surname'] for k, v in players.items()}
tournaments_names = {v['id']: v['name'] for k, v in tournaments.items()}
teams_names = get_team_names(results)  

In [7]:
def get_data_teams(results):
    data_teams = {}
    
    for tournament_id, tournament_teams in results.items():
        tournament_results = {}
        for team in tournament_teams:
            mask = team.get('mask')
            if mask is None or not all(c in '01' for c in mask) or not team['teamMembers']:
                continue

            team_id = team['team']['id']        
            tournament_results[team_id] = {}
            tournament_results[team_id]['mask'] = [int(c) for c in mask]
            tournament_results[team_id]['players'] = [player['player']['id'] for player in team['teamMembers']]
    
        masks_len = [len(team['mask']) for team in tournament_results.values()]
        if masks_len and masks_len.count(masks_len[0]) == len(masks_len):  
            data_teams[tournament_id] = tournament_results
        
    return data_teams

In [8]:
def split_data(data, train_id, test_id):
    train = {}
    test = {}
    for tournament_id, tournament_results in data.items():
        if tournament_id in train_id:
            train[tournament_id] = tournament_results
        elif tournament_id in test_id:
            test[tournament_id] = tournament_results
    return train, test

In [9]:
data_teams = get_data_teams(results)
train_teams, test_teams = split_data(data_teams, train_tournaments, test_tournaments)

In [10]:
def get_train_players(train_teams):
    train_players = []

    last_question = 0
    for tournament_id, tournament_teams in train_teams.items():
        for team_id, team_line_up in tournament_teams.items():
            questions = np.arange(last_question, last_question + len(team_line_up['mask']))
            for member in team_line_up['players']:
                for i, answer in enumerate(team_line_up['mask']):
                    train_players.append({'tournament_id': tournament_id, 
                                          'team_id': team_id, 
                                          'player_id': member,
                                          'question_id': questions[i],
                                          'answer': answer})
        last_question += len(team_line_up['mask'])
    
    return pd.DataFrame.from_dict(train_players)

In [11]:
def get_test_players(test_teams):
    test_players = []
    for tournament_id, tournament_teams in test_teams.items():
        for team_id, team_line_up in tournament_teams.items():
            for member in team_line_up['players']:
                test_players.append({'tournament_id': tournament_id, 
                                     'team_id': team_id, 
                                     'player_id': member,
                                     'question_id': -1,
                                     'true_answers': sum(team_line_up['mask'])})
    return pd.DataFrame.from_dict(test_players)

In [12]:
train_players = get_train_players(train_teams)
test_players = get_test_players(test_teams)

In [13]:
train_players.tail()

,tournament_id,team_id,player_id,question_id,answer
13749578,6191,76301,217859,28259,0
13749579,6191,76301,217859,28260,0
13749580,6191,76301,217859,28261,1
13749581,6191,76301,217859,28262,0
13749582,6191,76301,217859,28263,0


### 2

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [14]:
encoder = OneHotEncoder(handle_unknown='ignore')
X_train = encoder.fit_transform(train_players[['player_id', 'question_id']])
y_train = train_players['answer']

In [15]:
lr = linear_model.LogisticRegression(solver='liblinear', random_state=11)
lr.fit(X_train, y_train)

LogisticRegression(random_state=11, solver='liblinear')

In [16]:
def get_players_rating(data, preds, names):
    players_train = np.unique(data['player_id'])
    players_train_size = len(players_train)
    rating = pd.DataFrame({'player_id': players_train})
    rating['name'] = rating['player_id'].map(names)
    rating['power'] = preds[:players_train_size]
    return rating.sort_values(by='power', ascending=False).reset_index(drop=True)

In [46]:
get_players_rating(train_players, lr.coef_[0], players_names).head(20)

,player_id,name,power
0,27403,Максим Руссо,4.069043
1,4270,Александра Брутер,3.931775
2,28751,Иван Семушин,3.884502
3,27822,Михаил Савченков,3.853873
4,30270,Сергей Спешков,3.757432
5,30152,Артём Сорожкин,3.752641
6,18036,Михаил Левандовский,3.617950
7,20691,Станислав Мереминский,3.615595
8,87637,Антон Саксонов,3.541821
9,22799,Сергей Николенко,3.538993


### 3

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

In [18]:
questions_train = set(np.unique(train_players['question_id']))
X_test = encoder.transform(test_players[['player_id', 'question_id']])
predicts = lr.predict_proba(X_test)[:, 1]

In [56]:
def get_scores(test, predicts):
    test['predict'] = predicts
    test['score'] = test.groupby(['tournament_id', 'team_id'])['predict'].transform(lambda x: 1 - np.prod(1 - x))
    
    rating = test[['tournament_id', 'team_id', 'true_answers', 'score']].drop_duplicates().reset_index(drop=True)
    rating.sort_values(by=['tournament_id', 'true_answers'], ascending=False, inplace=True)
    rating['real_rank'] = rating.groupby('tournament_id')['true_answers'].transform(lambda x: np.arange(1, len(x) + 1))
    rating.sort_values(by=['tournament_id', 'score'], ascending=False, inplace=True)
    rating['predict_rank'] = rating.groupby('tournament_id')['score'].transform(lambda x: np.arange(1, len(x) + 1))

    spearmanr = rating.groupby('tournament_id').apply(lambda x: stats.spearmanr(x['real_rank'], x['predict_rank']).correlation).mean()
    kendalltau = rating.groupby('tournament_id').apply(lambda x: stats.kendalltau(x['real_rank'], x['predict_rank']).correlation).mean()
    return {'spearmanr': np.round(spearmanr, 6), 'kendalltau': np.round(kendalltau, 6)}

In [57]:
scores = get_scores(test_players, predicts)

In [58]:
print("Spearman's rank correlation coefficient: {}".format(scores['spearmanr']))
print("Kendall's coefficient of concordance: {}".format(scores['kendalltau']))

Spearman's rank correlation coefficient: 0.780468
Kendall's coefficient of concordance: 0.611758


### 4

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
- предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.

В прошлой логистической регрессии мы использовали тот факт, что если команда верно ответила на вопрос, то и игрок ответил верно, что не совсем верно. 

Предположим, что если команда не ответила верно на вопрос, то и каждый игрок из команды не ответил верно. И если игрок ответил верно на вопрос, то и его команда ответила верно.

У нас даны только ответы команд, поэтому наблюдаемые переменные - это ответы команд. Тогда скрытые переменные - ответы игроков. На E-шаге алгоритма находим скрытые переменные, на M-шаге обучаем логистическую регрессию на них же.

In [62]:
class LogisticRegression(object):      
    class Model(torch.nn.Module):
        def __init__(self, size):
            super().__init__()
            self.linear = torch.nn.Linear(size, 1)

        def forward(self, x):
            return torch.sigmoid(self.linear(x))
        
    def __init__(self, coef_, intercept_, learning_rate=5, epochs=25):
        size = coef_.shape[1]
        self.lr = self.Model(size)
        self.lr.linear.weight = torch.nn.parameter.Parameter(torch.as_tensor(coef_, dtype=torch.float32))
        self.lr.linear.bias.data.fill_(intercept_[0])
        self.criterion = torch.nn.BCELoss()
        self.optimizer = torch.optim.SGD(self.lr.parameters(), lr=learning_rate, momentum=0.9)
        self.epochs = epochs
    
    def _csr_to_tensor(self, data):
        coo = data.tocoo()
        values = coo.data
        indices = np.vstack((coo.row, coo.col))
        i = torch.LongTensor(indices)
        v = torch.FloatTensor(values)
        shape = coo.shape
        return torch.sparse.FloatTensor(i, v, torch.Size(shape))
    
    def fit(self, X_train, y_train):
        X_train = self._csr_to_tensor(X_train)
        y_train = torch.as_tensor(y_train)
        y_train = y_train.unsqueeze(1)
        epochs = range(self.epochs)
        for epoch in epochs:
            self.lr.train()
            self.optimizer.zero_grad()
            y_preds = self.lr(X_train)
            loss = self.criterion(y_preds, y_train)
            loss.backward()
            self.optimizer.step()
        return self
    
    def predict_proba(self, X_train):
        X_train = self._csr_to_tensor(X_train)
        self.lr.eval()
        with torch.no_grad():
            return self.lr(X_train).detach().numpy()
    
    def save_state(self):
        return deepcopy(self.lr.state_dict())
    
    def load_state(self, state):
        self.lr.load_state_dict(deepcopy(state))
        
    def get_coef(self): 
        return (self.lr.linear.weight.detach().numpy()[0])

In [63]:
class EM(object):
    def _expectation_step(self, preds, params):
        params['power'] = 1 - preds
        power = params.groupby(['team_id', 'question_id']).agg({'power': 'prod'}).reset_index()
        power['power'] = 1 - power['power']
        power = params[['team_id', 'question_id']].merge(power)
        players_params = np.clip(preds / power['power'], 0, 1).values
        players_params[params['answer'] == 0] = 0
        return players_params

    def _maximization_step(self, lr, X_train, player_power):
        return self.lr.fit(X_train, player_power)
    
    def __init__(self, init_lr, n_iter=10):
        self.lr = LogisticRegression(coef_=init_lr.coef_, intercept_=init_lr.intercept_)
        self.n_iter = n_iter
        
    def fit(self, X_train, y_train, params, X_test, test_results):
        best_state = self.lr.save_state()
        best_spearmanr = -1
        best_kendalltau = -1
        for i in range(self.n_iter):
            preds = self.lr.predict_proba(X_train)
            preds = preds.ravel()
            player_power = self._expectation_step(preds, params)
            self.lr = self._maximization_step(lr, X_train, player_power)
            preds = self.lr.predict_proba(X_test)
            scores = get_scores(test_results, preds)
            kendalltau = scores['kendalltau']
            spearmanr = scores['spearmanr']
            print("Spearman's rank correlation coefficient: {}".format(spearmanr))
            print("Kendall's coefficient of concordance: {}".format(kendalltau))
            if kendalltau > best_kendalltau:
                best_kendalltau = kendalltau
                best_spearmanr = spearmanr
                best_state = self.lr.save_state()
        self.lr.load_state(best_state)
        
    def get_coef(self):
        return self.lr.get_coef()

In [64]:
em = EM(init_lr=lr)
em.fit(X_train, y_train, train_players, X_test, test_players)

Spearman's rank correlation coefficient: 0.784447
Kendall's coefficient of concordance: 0.616525
Spearman's rank correlation coefficient: 0.784806
Kendall's coefficient of concordance: 0.616941
Spearman's rank correlation coefficient: 0.784829
Kendall's coefficient of concordance: 0.616945
Spearman's rank correlation coefficient: 0.784763
Kendall's coefficient of concordance: 0.616819
Spearman's rank correlation coefficient: 0.784506
Kendall's coefficient of concordance: 0.61627
Spearman's rank correlation coefficient: 0.784364
Kendall's coefficient of concordance: 0.616116
Spearman's rank correlation coefficient: 0.784303
Kendall's coefficient of concordance: 0.616002
Spearman's rank correlation coefficient: 0.784415
Kendall's coefficient of concordance: 0.616169
Spearman's rank correlation coefficient: 0.784219
Kendall's coefficient of concordance: 0.615917
Spearman's rank correlation coefficient: 0.78439
Kendall's coefficient of concordance: 0.615972


In [55]:
rating = get_players_rating(train_players, em.get_coef(), players_names)
rating['answered_questions_number'] = rating['player_id'].map(train_players.groupby('player_id')['question_id'].count())
rating.head(20)

,player_id,name,power,answered_questions_number
0,206158,Екатерина Золотухина,4.863925,36
1,40411,Дмитрий Кудинов,4.676504,45
2,82688,Игорь Калгин,4.668765,36
3,141282,Руслан Семенеев,4.550612,36
4,38175,Максим Пилипенко,4.510303,36
5,27403,Максим Руссо,4.498446,1796
6,22474,Илья Немец,4.381283,75
7,4270,Александра Брутер,4.239171,2240
8,36910,Иван Эйхгольц,4.193479,72
9,21428,Вадим Молдавский,4.172491,72


Как видно из таблицы, EM-алгоритм добавил больше людей в топ, которые сыграли очень мало вопросов за все время.

### 5

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в $\href{http://db.chgk.info}{базе~вопросов~ЧГК}$ (это чисто техническое дело, тут никакого ML нету).

In [65]:
def get_tournaments_rating(data, preds, questions_train, names):
    rating = dict(zip(questions_train, preds))
    data['difficulty'] = data['question_id'].map(rating)
    data['tournament_name'] = data['tournament_id'].map(names)
    tournaments_rating = data[['tournament_name', 'question_id', 'difficulty']].drop_duplicates()
    tournaments_rating = pd.DataFrame(tournaments_rating.groupby('tournament_name')['difficulty'].mean())
    return tournaments_rating.sort_values(by='difficulty', ascending=False).reset_index()

In [66]:
tournaments_rating = get_tournaments_rating(train_players, 
                                            -em.get_coef()[-len(questions_train):], 
                                            questions_train, 
                                            tournaments_names)
tournaments_rating.head(20)

,tournament_name,difficulty
0,Чемпионат Санкт-Петербурга. Первая лига,4.415092
1,Угрюмый Ёрш,2.277125
2,Первенство правого полушария,2.046634
3,Воображаемый музей,1.958635
4,Записки охотника,1.776209
5,Кубок городов,1.685715
6,Ускользающая сова,1.665978
7,Знание – Сила VI,1.655574
8,Чемпионат России,1.541816
9,Чемпионат Минска. Лига А. Тур четвёртый,1.539090


In [67]:
tournaments_rating.tail(20)

,tournament_name,difficulty
578,Школьный Синхрон-lite. Выпуск 3.3,-1.580996
579,Синхрон-lite. Выпуск XXX,-1.608123
580,Лига вузов. IV тур,-1.612828
581,Школьный Синхрон-lite. Выпуск 3.1,-1.692236
582,Школьная лига. II тур.,-1.719491
583,Школьный Синхрон-lite. Выпуск 2.3,-1.729010
584,Межфакультетский кубок МГУ. Отбор №4,-1.733677
585,Второй тематический турнир имени Джоуи Триббиани,-1.754644
586,Малый кубок Физтеха,-1.814579
587,Школьная лига. III тур.,-1.883882


Из рейтинга вопросов по сложности видно, что на чемпионатах мира вопросы очень сложные, а на школьных очень легкие, что полностью соответствует действительности.